In [6]:
# Imports
import pandas as pd      # Used to produce the final results table
import requests          # Used to grab data from online sources
import geopandas as gdp  # Used to handle geojson data 
import tabulate          # (Note: tabulate is indirectly used by pandas. I include it here so its more clear that it has to be installed)

# Calculating land area for each country  
First we will grab coordinate data for each country from the provided url.  
Next we will read the the data as a GeoDataFrame using the geopandas library.  
The geopandas library will then be used to calculate the land area of each country using its built in methods.  

In [7]:
countries_coordinates = requests.get("https://datahub.io/core/geo-countries/_r/-/data/countries.geojson")  # Grab the data from the url
gdf = gdp.GeoDataFrame.from_features(countries_coordinates.json()["features"], crs='EPSG:4326')            # Read the data as a geodataframe using the EPSG:4326 coordinate system
gdf = gdf.set_index('ADMIN',drop = True)                                                                   # Set the column "ADMIN" as index so that the full country name is used as index
gdf.head()                                                                                                 # Take a look at the geodataframe

,geometry,ISO_A3,ISO_A2
ADMIN,,,
Aruba,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ...",ABW,AW
Afghanistan,"MULTIPOLYGON (((71.0498 38.40866, 71.05714 38....",AFG,AF
Angola,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...",AGO,AO
Anguilla,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",AIA,AI
Albania,"MULTIPOLYGON (((19.74777 42.5789, 19.74601 42....",ALB,AL


Now we will calculate the area of each county.  
We do this by first making a copy of the geodataframe, then changing the projection of the copy so that area is preserved correctly.  
Afterwards we can simply use ".areas" on the copied data frame to calculate the area of every country. 

In [8]:
gdf_2 = gdf.copy()                             # Make a copy of the geodataframe
gdf_2 = gdf_2.to_crs({'proj':'cea'})           # Change the copys projection, in this case I choose to use Lambert Cylindrical Equal 
areas = gdf_2.area/1000000                     # Calculate the area for every country in km^2

# Look at the areas of the countries we are interrested in
countries = ['United States of America','United Kingdom','Turkey','Thailand','Philippines','India'] 
areas[countries]                            

ADMIN
United States of America    9.464444e+06
United Kingdom              2.437825e+05
Turkey                      7.800802e+05
Thailand                    5.144535e+05
Philippines                 2.932375e+05
India                       3.151478e+06
dtype: float64

We can verify that our calculation is correct by comparing our results to wikipedia's listing of land areas (https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area):  

US: 9147593  
UK: 242741   
Turkey: 769632   
Thailand: 510890   
Philippines: 298170   
India: 2973190 

Unfortunately we see that there are some differences.  
Perhaps another map projection would give better results, but it is also possible that the way that land area is defined by wikipedia does not fully align with the coordinates that we have.  
Either way we can see that our results are reasonably close.

# Calculating number of PM10 Stations  
Next up we will grab the station data from the provided url.  
Also this we will read in as a geodataframe.

In [9]:
pm10_stations = requests.get("https://api.energyandcleanair.org/stations?country=GB,US,TR,PH,IN,TH&format=geojson")  # Grab the data from the url
pm10_gdf = gdp.GeoDataFrame.from_features(pm10_stations.json()["features"])                                          # Read the data as a geodataframe
pm10_gdf.info()                                                                                                      # Look at info

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6104 entries, 0 to 6103
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   geometry           6104 non-null   geometry
 1   infos              5033 non-null   object  
 2   last_scraped_data  0 non-null      object  
 3   source             6104 non-null   object  
 4   first_updated      0 non-null      object  
 5   gadm2_id           5397 non-null   object  
 6   gpw                0 non-null      object  
 7   pollutants         6050 non-null   object  
 8   last_updated       6104 non-null   object  
 9   name               6104 non-null   object  
 10  names              5177 non-null   object  
 11  city_id            6104 non-null   object  
 12  show_in_dashboard  6104 non-null   bool    
 13  timezone           5720 non-null   object  
 14  country_id         6104 non-null   object  
 15  gadm1_id           5946 non-null   object  
 16

The next step is to filter out all stations that do not measure pm10.  
To do this I have decided to use a simple function that checks if pm10 is included in the list of measured pollutants for each station

In [10]:
# Function to check if a station measures pm10
# Some stations dont have a proper list. I use try-except to return False instead of throwing an error.
def has_pm10(i): 
    try:                                         
        return 'pm10' in pm10_gdf.loc[i,'pollutants']
    except:
        return False

# Create a list with True for each station that has pm10 and False for all others        
stations_with_pm10 = [has_pm10(i) for i in pm10_gdf.index]

# Remove all stations from the dataframe that do not have pm10
pm10_gdf = pm10_gdf.iloc[stations_with_pm10]
pm10_gdf.info()    

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2252 entries, 0 to 6100
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   geometry           2252 non-null   geometry
 1   infos              1505 non-null   object  
 2   last_scraped_data  0 non-null      object  
 3   source             2252 non-null   object  
 4   first_updated      0 non-null      object  
 5   gadm2_id           1673 non-null   object  
 6   gpw                0 non-null      object  
 7   pollutants         2252 non-null   object  
 8   last_updated       2252 non-null   object  
 9   name               2252 non-null   object  
 10  names              1384 non-null   object  
 11  city_id            2252 non-null   object  
 12  show_in_dashboard  2252 non-null   bool    
 13  timezone           1919 non-null   object  
 14  country_id         2252 non-null   object  
 15  gadm1_id           2209 non-null   object  
 16  id 

We can see that we have reduced the dataframe by about 4000 rows by removing all non-pm10 stations.  
Next we can simply count how many pm10 stations there are by checking how often each country code appears in the 'country_id' column.  

In [11]:
pm10_stations_per_county = pm10_gdf['country_id'].value_counts()  # Count how often each country code appears
pm10_stations_per_county                                          # Look at he counts

country_id
US    1035
IN     536
TR     332
GB     165
TH     162
PH      22
Name: count, dtype: int64

# Putting together the results table  
Finally we round things of by putting the results table together and calculating the density of pm10 stations.

In [ ]:
results = pd.DataFrame(index = countries,  columns = ['Number of PM10 Stations','Area (in square kilometers)','Density of PM10 Stations per 1000 sq. km']) # Create the dataframe
results.index.name = "Country Name"                                                                                                                        # Set the index name to "Country Name"
results['Number of PM10 Stations'] = pm10_stations_per_county.rename(index={'US': 'United States of America',                                              # Add number of pm10 stations, but replace country ids with full country names
                                                                            'GB': 'United Kingdom',
                                                                            'IN': 'India',
                                                                            'TR': 'Turkey',
                                                                            'TH': 'Thailand',
                                                                            'PH': 'Philippines'})
results['Area (in square kilometers)'] = areas                                                                                                             # Add country area to table
results['Density of PM10 Stations per 1000 sq. km'] = 1000 * results['Number of PM10 Stations'] / results['Area (in square kilometers)']                   # Calculate the station density
results = results.sort_values(by = 'Density of PM10 Stations per 1000 sq. km', ascending = False)                                                          # Sort the table after station density in descending order
print(results.to_markdown(floatfmt=(".0f", ".0f", ".0f",".3f")))                                                                                           # Print the result table (with the right amount of decimals per column) 

| Country Name             |   Number of PM10 Stations |   Area (in square kilometers) |   Density of PM10 Stations per 1000 sq. km |
|:-------------------------|--------------------------:|------------------------------:|-------------------------------------------:|
| United Kingdom           |                       165 |                        243783 |                                      0.677 |
| Turkey                   |                       332 |                        780080 |                                      0.426 |
| Thailand                 |                       162 |                        514453 |                                      0.315 |
| India                    |                       536 |                       3151478 |                                      0.170 |
| United States of America |                      1035 |                       9464444 |                                      0.109 |
| Philippines              |                        22 |      